# Final Project - Shawn Feng, Will Novak, Eddie Loyd

In [1]:
import os
import pretty_midi
import numpy as np
import fluidsynth
import glob
import tensorflow as tf
from IPython.display import display, Audio

_SAMPLING_RATE = 16000

filenames = glob.glob('midi_data/arpeggios/**/*.mid', recursive=True)
sample_file = filenames[0]  # Change this to the desired MIDI file

pm = pretty_midi.PrettyMIDI(sample_file)

# Create a FluidSynth synthesizer from pretty_midi
fs = pm.fluidsynth(fs=_SAMPLING_RATE)

# Generate the waveform
waveform = fs.get_waveform()

# Display the audio
display(Audio(waveform, rate=_SAMPLING_RATE))

AttributeError: module 'fluidsynth' has no attribute 'Synth'

In [ ]:
numpy_array.shape